In [1]:
import pandas as pd

df = pd.read_csv("../data/game_results.csv")

In [2]:
df = df[['id', 'season', 'week', 'home_points','away_points','home_team','away_team','home_id','away_id']]
df.head()

,id,season,week,home_points,away_points,home_team,away_team,home_id,away_id
0,212350097,2001,1,45,24,Louisville,New Mexico State,97,166
1,63770,2001,1,21,7,Nebraska,TCU,158,2628
2,212370275,2001,1,26,17,Wisconsin,Virginia,275,258
3,212370252,2001,1,70,35,BYU,Tulane,252,2655
4,212370201,2001,1,10,0,Oklahoma,North Carolina,201,153


In [3]:
df['winner'] = df['home_id'][df['home_points'] > df['away_points']]

In [6]:
df['winner'][df['home_points'] < df['away_points']] = df['away_id']
df['winner'] = df['winner'].astype(int)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [7]:
df.iloc[:10, :]

,id,season,week,home_points,away_points,home_team,away_team,home_id,away_id,winner
0,212350097,2001,1,45,24,Louisville,New Mexico State,97,166,97
1,63770,2001,1,21,7,Nebraska,TCU,158,2628,158
2,212370275,2001,1,26,17,Wisconsin,Virginia,275,258,275
3,212370252,2001,1,70,35,BYU,Tulane,252,2655,252
4,212370201,2001,1,10,0,Oklahoma,North Carolina,201,153,201
5,212380183,2001,1,7,13,Syracuse,Georgia Tech,183,59,59
6,212380038,2001,1,22,24,Colorado,Fresno State,38,278,278
7,212420218,2001,2,45,26,Temple,Navy,218,2426,218
8,212422649,2001,2,38,7,Toledo,Minnesota,2649,135,2649
9,212422006,2001,2,31,29,Akron,Ohio,2006,195,2006


In [119]:
import numpy as np
df2 = pd.read_csv("../data/game_stats.csv")
df2.columns

Index(['Unnamed: 0', 'id', 'home.school', 'home.conference', 'home.points',
       'home.fumblesRecovered', 'home.rushingTDs', 'home.puntReturnYards',
       'home.puntReturnTDs', 'home.puntReturns', 'home.passingTDs',
       'home.kickingPoints', 'home.firstDowns', 'home.thirdDownEff',
       'home.fourthDownEff', 'home.totalYards', 'home.netPassingYards',
       'home.completionAttempts', 'home.yardsPerPass', 'home.rushingYards',
       'home.rushingAttempts', 'home.yardsPerRushAttempt',
       'home.totalPenaltiesYards', 'home.turnovers', 'home.fumblesLost',
       'home.interceptions', 'home.possessionTime', 'away.school',
       'away.conference', 'away.points', 'away.fumblesRecovered',
       'away.rushingTDs', 'away.puntReturnYards', 'away.puntReturnTDs',
       'away.puntReturns', 'away.passingTDs', 'away.kickingPoints',
       'away.interceptionYards', 'away.interceptionTDs',
       'away.passesIntercepted', 'away.firstDowns', 'away.thirdDownEff',
       'away.fourthDownEff', 

In [162]:
def process_third(x):
    if(str(x).split('-')[-1] == '0'): return 1
    else: return float(str(x).split('-')[0])/float(str(x).split('-')[1])

stats = df2[['id', 'home.school','home.points', 'home.thirdDownEff','home.completionAttempts', 'home.yardsPerPass','home.yardsPerRushAttempt','home.totalPenaltiesYards','home.turnovers','home.possessionTime','away.school','away.points','away.thirdDownEff','away.completionAttempts', 'away.yardsPerPass','away.yardsPerRushAttempt','away.totalPenaltiesYards','away.turnovers','away.possessionTime','home.rushingAttempts','away.rushingAttempts','Year','Week']]
stats.dropna(inplace = True)
stats.loc[:,'home.pointDiff'] = df2['home.points'] - df2['away.points']
stats.loc[:,'away.pointDiff'] = -stats['home.pointDiff']
stats.loc[:,'home.completionAttempts'] = df2['home.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'away.completionAttempts'] = df2['away.completionAttempts'].apply(lambda x: float(str(x).split('-')[-1]))
stats.loc[:,'home.yardsAllowedPerPlay'] = df2['away.totalYards']/(stats['away.completionAttempts'] + df2['away.rushingAttempts'])
stats.loc[:,'away.yardsAllowedPerPlay'] = df2['home.totalYards']/(stats['home.completionAttempts'] + df2['home.rushingAttempts'])
stats.loc[:,'home.yards_per_play'] = stats['away.yardsAllowedPerPlay']
stats.loc[:,'away.yards_per_play'] = stats['home.yardsAllowedPerPlay']
stats.loc[:,'home.forcedTO'] = df2['away.turnovers']
stats.loc[:,'away.forcedTO'] = df2['home.turnovers']
stats.loc[:,'home.passAttemptsAllowed'] = stats['away.completionAttempts']
stats.loc[:,'away.passAttemptsAllowed'] = stats['home.completionAttempts']
stats.loc[:,'home.yardsAllowedPerRush'] = df2['away.yardsPerRushAttempt']
stats.loc[:,'away.yardsAllowedPerRush'] = df2['home.yardsPerRushAttempt']
stats.loc[:,'home.thirdDownEff'] = stats.loc[:,'home.thirdDownEff'].apply(process_third)
stats.loc[:,'away.thirdDownEff'] = stats.loc[:,'away.thirdDownEff'].apply(process_third)
stats.loc[:,'home.totalPenaltiesYards'] = stats.loc[:,'home.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'away.totalPenaltiesYards'] = stats.loc[:,'away.totalPenaltiesYards'].apply(lambda x: float(x.split('-')[-1]))
stats.loc[:,'home.possessionTime'] = stats['home.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats.loc[:,'away.possessionTime'] = stats['away.possessionTime'].apply(lambda x: float(x.split(':')[0])*60 + float(x.split(':')[1]))
stats.iloc[:10,:]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,home.school,home.points,home.thirdDownEff,home.completionAttempts,home.yardsPerPass,home.yardsPerRushAttempt,home.totalPenaltiesYards,home.turnovers,home.possessionTime,away.school,away.points,away.thirdDownEff,away.completionAttempts,away.yardsPerPass,away.yardsPerRushAttempt,away.totalPenaltiesYards,away.turnovers,away.possessionTime,home.rushingAttempts,away.rushingAttempts,Year,Week,home.pointDiff,away.pointDiff,home.yardsAllowedPerPlay,away.yardsAllowedPerPlay,home.yards_per_play,away.yards_per_play,home.forcedTO,away.forcedTO,home.passAttemptsAllowed,away.passAttemptsAllowed,home.yardsAllowedPerRush,away.yardsAllowedPerRush
0,242410259,Virginia Tech,13,0.384615,29.0,5.3,4.1,0.0,3.0,1170.0,USC,24,0.428571,29.0,9.4,3.5,0.0,0.0,1028.0,34.0,29.0,2004,1,-11,11,6.431034,4.666667,4.666667,6.431034,0.0,3.0,29.0,29.0,3.5,4.1
1,242410193,Miami (OH),49,0.363636,37.0,7.9,4.2,0.0,3.0,1085.0,Indiana State,0,1.000000,33.0,4.8,1.4,63.0,3.0,1108.0,39.0,31.0,2004,1,49,-49,3.187500,5.973684,5.973684,3.187500,3.0,3.0,33.0,37.0,1.4,4.2
2,242482426,Navy,27,0.400000,9.0,14.3,5.3,0.0,3.0,1123.0,Duke,12,0.153846,22.0,4.1,4.5,0.0,0.0,1077.0,57.0,39.0,2004,2,15,-15,4.344262,6.515152,6.515152,4.344262,0.0,3.0,22.0,9.0,4.5,5.3
3,242480120,Maryland,23,0.416667,22.0,7.7,4.3,0.0,4.0,1095.0,Northern Illinois,20,0.222222,37.0,5.9,3.1,0.0,3.0,1114.0,46.0,38.0,2004,2,3,-3,4.493333,5.397059,5.397059,4.493333,3.0,4.0,37.0,22.0,3.1,4.3
4,242480153,North Carolina,49,0.833333,24.0,9.8,7.6,0.0,4.0,1059.0,William & Mary,38,1.000000,41.0,7.9,3.9,45.0,0.0,1141.0,45.0,31.0,2004,2,11,-11,6.138889,8.333333,8.333333,6.138889,0.0,4.0,41.0,24.0,3.9,7.6
5,242480228,Clemson,37,0.312500,41.0,7.0,2.4,0.0,3.0,1140.0,Wake Forest,30,0.444444,25.0,6.2,4.7,0.0,1.0,1020.0,35.0,54.0,2004,2,7,-7,5.189873,4.881579,4.881579,5.189873,1.0,3.0,25.0,41.0,4.7,2.4
6,242480152,NC State,42,0.384615,32.0,7.2,4.7,0.0,1.0,1231.0,Richmond,0,1.000000,27.0,2.3,2.9,77.0,3.0,1052.0,37.0,36.0,2004,2,42,-42,2.650794,5.840580,5.840580,2.650794,3.0,1.0,27.0,32.0,2.9,4.7
7,242480164,Rutgers,19,0.380952,34.0,7.1,3.5,0.0,1.0,1443.0,Michigan State,14,0.166667,39.0,6.9,4.1,0.0,3.0,737.0,50.0,22.0,2004,2,5,-5,5.901639,4.940476,4.940476,5.901639,3.0,1.0,39.0,34.0,4.1,3.5
8,242480356,Illinois,52,0.454545,23.0,11.2,6.2,0.0,0.0,1114.0,Florida A&M,13,1.000000,55.0,5.2,2.8,36.0,3.0,1086.0,48.0,25.0,2004,2,39,-39,4.475000,7.802817,7.802817,4.475000,3.0,0.0,55.0,23.0,2.8,6.2
9,242480275,Wisconsin,34,0.428571,20.0,8.5,4.7,0.0,1.0,1172.0,UCF,6,0.375000,24.0,7.4,1.5,0.0,3.0,1033.0,49.0,35.0,2004,2,28,-28,3.898305,5.797101,5.797101,3.898305,3.0,1.0,24.0,20.0,1.5,4.7


In [216]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
home_index = [x for x in list(stats.columns) if 'home.' in x] + ['Week', 'Year', 'id']
home = stats[home_index].drop('home.points',axis = 1)
home = home.groupby(['Year', 'home.school','Week']).mean()
home.iloc[:50,:]

home.thirdDownEff  home.completionAttempts  \
Year home.school    Week                                               
2004 Air Force      2              0.555556                     18.0   
                    3              0.600000                     16.0   
                    6              0.473684                     24.0   
                    7              0.357143                      9.0   
                    9              0.266667                     24.0   
                    12             0.428571                     20.0   
                    13             0.642857                     19.0   
     Akron          3              0.416667                     44.0   
                    7              0.416667                     23.0   
                    9              0.461538                     33.0   
                    11             0.250000                     45.0   
                    13             0.400000                     43.0   
     Alabama        2              0.333333                     22.0   
                    3              0.583333                     22.0   
                    4              0.428571                     31.0   
                    6              0.000000                     24.0   
                    8              0.384615                      9.0   
                    11             0.250000                     15.0   
                    13             0.214286                     30.0   
     Arizona        2              0.500000                     23.0   
                    3              0.416667                     25.0   
                    4              0.181818                     15.0   
                    5              0.368421                     31.0   
                    9              0.200000                     32.0   
                    10             0.200000                     38.0   
                    14             0.333333                     32.0   
     Arizona State  2              0.312500                     44.0   
                    4              0.631579                     46.0   
                    5              0.266667                     40.0   
                    9              0.476190                     51.0   
                    11             0.294118                     43.0   
                    12             0.368421                     43.0   
     Arkansas       2              0.625000                     24.0   
                    3              0.470588                     29.0   
                    5              0.250000                     12.0   
                    9              0.153846                     21.0   
                    12             0.562500                     20.0   
                    14             0.400000                     32.0   
     Arkansas State 4              0.533333                     41.0   
                    8              0.312500                     23.0   
                    12             0.125000                     23.0   
                    13             0.454545                     24.0   
     Army           3              0.214286                     32.0   
                    6              0.187500                     26.0   
                    7              0.642857                     21.0   
                    11             0.437500                     38.0   
                    13             0.294118                     46.0   
     Auburn         2              0.600000                     24.0   
                    4              0.307692                     27.0   
                    5              0.545455                     32.0   

                          home.yardsPerPass  home.yardsPerRushAttempt  \
Year home.school    Week                                                
2004 Air Force      2                   4.9                       3.4   
                    3                   9.1                   

In [217]:
aggstats = home.copy()
idx = list(aggstats.index)
n = 1
for i,x in enumerate(idx):
    if(i == 0): continue
    if(x[1] == idx[i-1][1]):
        aggstats.loc[(x[0],x[1],x[2]),:] = aggstats.loc[(x[0],x[1],x[2]),:] + aggstats.loc[(idx[i-1][0],idx[i-1][1],idx[i-1][2]),:]*n
        aggstats.loc[(x[0],x[1],x[2]),:] = aggstats.loc[(x[0],x[1],x[2]),:]/(n+1)
        n += 1
    else: n = 1;
# for year in idxs[0]:
#     for team in idxs[1]:
#         for week in idxs[2]:
#             it = week - 1
#             n = 1
#             while it > 0:
#                 try:
#                     aggstats.loc[(year,team,week),:] = aggstats.loc[(year,team,week),:] + aggstats.loc[(year,team,it),:]
#                     n += 1
#                 except KeyError: pass
#                 it -= 1
#             try:
#                 aggstats.loc[(year,team,week),:] = aggstats.loc[(year,team,week),:]/n
#             except KeyError: pass
            
aggstats.iloc[:50,:]



home.thirdDownEff  home.completionAttempts  \
Year home.school    Week                                               
2004 Air Force      2              0.555556                18.000000   
                    3              0.577778                17.000000   
                    6              0.543080                19.333333   
                    7              0.496596                16.750000   
                    9              0.450610                18.200000   
                    12             0.446937                18.500000   
                    13             0.474925                18.571429   
     Akron          3              0.416667                44.000000   
                    7              0.416667                33.500000   
                    9              0.431624                33.333333   
                    11             0.386218                36.250000   
                    13             0.388974                37.600000   
     Alabama        2              0.333333                22.000000   
                    3              0.458333                22.000000   
                    4              0.448413                25.000000   
                    6              0.336310                24.750000   
                    8              0.345971                21.600000   
                    11             0.329976                20.500000   
                    13             0.313448                21.857143   
     Arizona        2              0.500000                23.000000   
                    3              0.458333                24.000000   
                    4              0.366162                21.000000   
                    5              0.366726                23.500000   
                    9              0.333381                25.200000   
                    10             0.311151                27.333333   
                    14             0.314320                28.000000   
     Arizona State  2              0.312500                44.000000   
                    4              0.472039                45.000000   
                    5              0.403582                43.333333   
                    9              0.421734                45.250000   
                    11             0.396211                44.800000   
                    12             0.391579                44.500000   
     Arkansas       2              0.625000                24.000000   
                    3              0.547794                26.500000   
                    5              0.448529                21.666667   
                    9              0.374859                21.500000   
                    12             0.412387                21.200000   
                    14             0.410322                23.000000   
     Arkansas State 4              0.533333                41.000000   
                    8              0.422917                32.000000   
                    12             0.323611                29.000000   
                    13             0.356345                27.750000   
     Army           3              0.214286                32.000000   
                    6              0.200893                29.000000   
                    7              0.348214                26.333333   
                    11             0.370536                29.250000   
                    13             0.355252                32.600000   
     Auburn         2              0.600000                24.000000   
                    4              0.453846                25.500000   
                    5              0.484382                27.666667   

                          home.yardsPerPass  home.yardsPerRushAttempt  \
Year home.school    Week                                                
2004 Air Force      2              4.900000                  3.400000   
                    3              7.000000                  5